Pressure value imputer using KNN 

In [1]:
!pip install impyute
!pip install xlwt
!pip install openpyxl

##1. Importing required Dependencies



*   Pandas - Dataframe Handler
*   Numpy - Large Array computation
*   Fancyimpute - Imputation algorithm
*   Sklearn - Machine Learning Functions
* Matplotlib - Visualising Library
* Seaborn - Highly visualizing library


In [ ]:
import pandas as pd
import numpy as np
import impyute as impy
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score 

import matplotlib.pyplot as plt
import seaborn as sns

Mounting Google Drive

In [29]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/piv-project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Preparing Data from the Google Drive

**Dropping the X and Y Co-ordinates and labels**

In [ ]:

data = pd.read_excel('/content/drive/My Drive/piv-project/pressure-data.xlsx',
                     header=None, sheet_name='Side')
data = data.drop(labels=0, axis='columns')
data = data.drop(labels=[0,1,2],  axis='rows')


Dataframe View with first 5 rows of the dataframe

In [ ]:
data.to_csv('pressure-actual-back-values.csv')

Introducing NaN values in sensor value data (data columns) 

In [ ]:
nan_data = data.copy() # Creating a deep copy of dataframe

In [ ]:
# Making the below list of columns name [7,24,18,29,42,47,3] as NaN values
for i in [7,24,18,29,42,47,3]:
  nan_data [i] = pd.DataFrame([np.NaN for _ in range(data[i].size)])

In [50]:
nan_data.head() # Viewing the dataframe after NaN value introduction

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
3,-0.73543,-0.71443,NaN,-0.43799,-0.38768,-0.81174,NaN,-0.75074,-0.63518,-0.43624,-0.81760,-0.91208,-0.74679,-0.54795,-0.39211,-0.75015,-0.75262,NaN,-0.38594,-0.43767,-0.66134,-0.64342,-0.65063,NaN,-0.63227,-0.70803,-0.73730,-0.82288,NaN,-0.69694,-0.76877,-0.75906,-0.77013,-0.82896,-0.77228,-0.73535,-0.81207,-0.86305,-1.18611,-1.36653,-0.74524,NaN,-0.82351,-0.85456,-1.61990,-0.75199,NaN,-0.79543,-0.79823,-0.81202
4,-0.73448,-0.68413,NaN,-0.39831,-0.42358,-0.78511,NaN,-0.76845,-0.49411,-0.51157,-0.81100,-0.90097,-0.89930,-0.49715,-0.38384,-0.73156,-0.75467,NaN,-0.24126,-0.43266,-0.64765,-0.63686,-0.65063,NaN,-0.68474,-0.70444,-0.69128,-0.79535,NaN,-0.65605,-0.79040,-0.74803,-0.74344,-0.76042,-0.72919,-0.73450,-0.80882,-0.81292,-1.14635,-1.12323,-0.76246,NaN,-0.82928,-0.83638,-1.45437,-0.77504,NaN,-0.77490,-0.76340,-0.74568
5,-0.81078,-0.65934,NaN,-0.44945,-0.25743,-0.81529,NaN,-0.88262,-0.43992,-0.50342,-0.81430,-0.86866,-1.03655,-0.38417,-0.46156,-0.73871,-0.76487,NaN,-0.13445,-0.41962,-0.67321,-0.65652,-0.65165,NaN,-0.60502,-0.70803,-0.70936,-0.76506,NaN,-0.66227,-0.81202,-0.75815,-0.74522,-0.70426,-0.70137,-0.77114,-0.79500,-0.81556,-1.13522,-0.89756,-0.76821,NaN,-0.85717,-0.83638,-1.41703,-0.78187,NaN,-0.81391,-0.74847,-0.77736
6,-0.86447,-0.68780,NaN,-0.38156,-0.37348,-0.86498,NaN,-0.95697,-0.24809,-0.48510,-0.81430,-0.86664,-1.06167,-0.19060,-0.44420,-0.75373,-0.74854,NaN,-0.22378,-0.30531,-0.68325,-0.66495,-0.67297,NaN,-0.52329,-0.73679,-0.70196,-0.72835,NaN,-0.68450,-0.80687,-0.78022,-0.69897,-0.65857,-0.62777,-0.78137,-0.77225,-0.78214,-1.00166,-0.71861,-0.77231,NaN,-0.84563,-0.88928,-1.17782,-0.77760,NaN,-0.79851,-0.71563,-0.72489
7,-0.88614,-0.87602,NaN,-0.37627,-0.32840,-0.86765,NaN,-0.94370,-0.26443,-0.31511,-0.80275,-0.89188,-1.00067,-0.18447,-0.32266,-0.74658,-0.76385,NaN,-0.44128,-0.09273,-0.66864,-0.67805,-0.66688,NaN,-0.46476,-0.72241,-0.70114,-0.69990,NaN,-0.70583,-0.78319,-0.81516,-0.66161,-0.61003,-0.54520,-0.76262,-0.76332,-0.73906,-0.91023,-0.65073,-0.75918,NaN,-0.83794,-0.97689,-1.15663,-0.74943,NaN,-0.75539,-0.69971,-0.73479


##2. Defining functions for Data Imputation and Error Function

In [ ]:
n = 1000 # Number of time series to be predicted 

k = 3 # K Number of neighbours

In [ ]:
def knn_impute(data_frame):
  imputed_ls = list() # for storing imputed time series record
  for i in range(n): # 100 specifies the number of time series value 
    # Selecting the row i
    row_val = data_frame.iloc[i,:] 
    #Reshaping the sensor data into a array of 10x5
    row_np = row_val.values.reshape(10,5)
    #Imputing the missing values using fast_KNN imputer
    row_imputed = impy.fast_knn(row_np,k=k)
    #Reshaping again to 1x50 , to make it just like the original dataset
    row_imputed = row_imputed.reshape(1,50)
    #Appending into the list imputed_ls
    imputed_ls.append(row_imputed.tolist())
    
  #converting List to pandas dataframe
  imputed_df = pd.DataFrame(imputed_ls[i][0] for i in range(n))
  
  return imputed_df.copy()

In [ ]:
def err_cal(acc_df, imp_df):
  mse_ls = list()
  r_sqr = list()
  for i in range(n):
    acc_val = list( acc_df.iloc[i,:]) # iloc, returns the values at the specific list of columns
    imp_val = list(imp_df.iloc[i, :])
    mse_val = mse(acc_val, imp_val)  # calculates the mse value of the entire missing value at a time 
    r2_val = r2_score(acc_val, imp_val)
    mse_ls.append(mse_val)
    r_sqr.append(r2_val)
    #appending mse_value to the mse_list 
  #List to Dataframe 
  mse_col = pd.Series(mse_ls,name='mse')
  r2_col = pd.Series(r_sqr, name='r2_error')
  mse_df = pd.concat([pd.Series(range(n),name='time'),mse_col,r2_col],axis=1)
  return mse_df
  

Imputing the dataframe 

In [54]:
imp_df = knn_impute(nan_data.copy())

imp_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,-0.73543,-0.71443,-0.753096,-0.43799,-0.38768,-0.81174,-0.783000,-0.75074,-0.63518,-0.43624,-0.81760,-0.91208,-0.74679,-0.54795,-0.39211,-0.75015,-0.75262,-0.752948,-0.38594,-0.43767,-0.66134,-0.64342,-0.65063,-0.784595,-0.63227,-0.70803,-0.73730,-0.82288,-0.772340,-0.69694,-0.76877,-0.75906,-0.77013,-0.82896,-0.77228,-0.73535,-0.81207,-0.86305,-1.18611,-1.36653,-0.74524,-0.770785,-0.82351,-0.85456,-1.61990,-0.75199,-0.686030,-0.79543,-0.79823,-0.81202
1,-0.73448,-0.68413,-0.833220,-0.39831,-0.42358,-0.78511,-0.770345,-0.76845,-0.49411,-0.51157,-0.81100,-0.90097,-0.89930,-0.49715,-0.38384,-0.73156,-0.75467,-0.827810,-0.24126,-0.43266,-0.64765,-0.63686,-0.65063,-0.731403,-0.68474,-0.70444,-0.69128,-0.79535,-0.723723,-0.65605,-0.79040,-0.74803,-0.74344,-0.76042,-0.72919,-0.73450,-0.80882,-0.81292,-1.14635,-1.12323,-0.76246,-0.762039,-0.82928,-0.83638,-1.45437,-0.77504,-0.666838,-0.77490,-0.76340,-0.74568
2,-0.81078,-0.65934,-0.915767,-0.44945,-0.25743,-0.81529,-0.735754,-0.88262,-0.43992,-0.50342,-0.81430,-0.86866,-1.03655,-0.38417,-0.46156,-0.73871,-0.76487,-0.912329,-0.13445,-0.41962,-0.67321,-0.65652,-0.65165,-0.703546,-0.60502,-0.70803,-0.70936,-0.76506,-0.694352,-0.66227,-0.81202,-0.75815,-0.74522,-0.70426,-0.70137,-0.77114,-0.79500,-0.81556,-1.13522,-0.89756,-0.76821,-0.764899,-0.85717,-0.83638,-1.41703,-0.78187,-0.692451,-0.81391,-0.74847,-0.77736
3,-0.86447,-0.68780,-0.799350,-0.38156,-0.37348,-0.86498,-0.759794,-0.95697,-0.24809,-0.48510,-0.81430,-0.86664,-1.06167,-0.19060,-0.44420,-0.75373,-0.74854,-0.960885,-0.22378,-0.30531,-0.68325,-0.66495,-0.67297,-0.508629,-0.52329,-0.73679,-0.70196,-0.72835,-0.640181,-0.68450,-0.80687,-0.78022,-0.69897,-0.65857,-0.62777,-0.78137,-0.77225,-0.78214,-1.00166,-0.71861,-0.77231,-0.767281,-0.84563,-0.88928,-1.17782,-0.77760,-0.752098,-0.79851,-0.71563,-0.72489
4,-0.88614,-0.87602,-0.906754,-0.37627,-0.32840,-0.86765,-0.825554,-0.94370,-0.26443,-0.31511,-0.80275,-0.89188,-1.00067,-0.18447,-0.32266,-0.74658,-0.76385,-0.790394,-0.44128,-0.09273,-0.66864,-0.67805,-0.66688,-0.632410,-0.46476,-0.72241,-0.70114,-0.69990,-0.614551,-0.70583,-0.78319,-0.81516,-0.66161,-0.61003,-0.54520,-0.76262,-0.76332,-0.73906,-0.91023,-0.65073,-0.75918,-0.745980,-0.83794,-0.97689,-1.15663,-0.74943,-0.765008,-0.75539,-0.69971,-0.73479


Error value calculation

In [55]:
err_df = err_cal(data.copy(), imp_df.copy())
err_df
avg_mse = err_df['mse'].values.mean()
avg_r2 = err_df['r2_error'].values.mean()
with open('knn_result_analysis_front_04022020.txt','a') as fi_le:
  fi_le.write('No. of Sample Time Series : {} \n K- Value : {} \n Average MSE : {} \n Average RSE : {} \n -----------------------------------'.format(n,k,avg_mse,avg_r2))
print("Number of Sample Time series :",n)
print("KNN , K-Value :",k)
print("Average MSE :",avg_mse)
print("Average R2", avg_r2)

Number of Sample Time series : 1000
KNN , K-Value : 3
Average MSE : 0.005485204953502853
Average R2 0.8976097281273906


##Saving the result to csv and excel

In [ ]:
pd_save = pd.concat([imp_df,err_df],axis=1)

In [57]:
pd_save.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,time,mse,r2_error
0,-0.73543,-0.71443,-0.753096,-0.43799,-0.38768,-0.81174,-0.783000,-0.75074,-0.63518,-0.43624,-0.81760,-0.91208,-0.74679,-0.54795,-0.39211,-0.75015,-0.75262,-0.752948,-0.38594,-0.43767,-0.66134,-0.64342,-0.65063,-0.784595,-0.63227,-0.70803,-0.73730,-0.82288,-0.772340,-0.69694,-0.76877,-0.75906,-0.77013,-0.82896,-0.77228,-0.73535,-0.81207,-0.86305,-1.18611,-1.36653,-0.74524,-0.770785,-0.82351,-0.85456,-1.61990,-0.75199,-0.686030,-0.79543,-0.79823,-0.81202,0,0.000640,0.985628
1,-0.73448,-0.68413,-0.833220,-0.39831,-0.42358,-0.78511,-0.770345,-0.76845,-0.49411,-0.51157,-0.81100,-0.90097,-0.89930,-0.49715,-0.38384,-0.73156,-0.75467,-0.827810,-0.24126,-0.43266,-0.64765,-0.63686,-0.65063,-0.731403,-0.68474,-0.70444,-0.69128,-0.79535,-0.723723,-0.65605,-0.79040,-0.74803,-0.74344,-0.76042,-0.72919,-0.73450,-0.80882,-0.81292,-1.14635,-1.12323,-0.76246,-0.762039,-0.82928,-0.83638,-1.45437,-0.77504,-0.666838,-0.77490,-0.76340,-0.74568,1,0.000745,0.980124
2,-0.81078,-0.65934,-0.915767,-0.44945,-0.25743,-0.81529,-0.735754,-0.88262,-0.43992,-0.50342,-0.81430,-0.86866,-1.03655,-0.38417,-0.46156,-0.73871,-0.76487,-0.912329,-0.13445,-0.41962,-0.67321,-0.65652,-0.65165,-0.703546,-0.60502,-0.70803,-0.70936,-0.76506,-0.694352,-0.66227,-0.81202,-0.75815,-0.74522,-0.70426,-0.70137,-0.77114,-0.79500,-0.81556,-1.13522,-0.89756,-0.76821,-0.764899,-0.85717,-0.83638,-1.41703,-0.78187,-0.692451,-0.81391,-0.74847,-0.77736,2,0.001207,0.971643
3,-0.86447,-0.68780,-0.799350,-0.38156,-0.37348,-0.86498,-0.759794,-0.95697,-0.24809,-0.48510,-0.81430,-0.86664,-1.06167,-0.19060,-0.44420,-0.75373,-0.74854,-0.960885,-0.22378,-0.30531,-0.68325,-0.66495,-0.67297,-0.508629,-0.52329,-0.73679,-0.70196,-0.72835,-0.640181,-0.68450,-0.80687,-0.78022,-0.69897,-0.65857,-0.62777,-0.78137,-0.77225,-0.78214,-1.00166,-0.71861,-0.77231,-0.767281,-0.84563,-0.88928,-1.17782,-0.77760,-0.752098,-0.79851,-0.71563,-0.72489,3,0.000908,0.978394
4,-0.88614,-0.87602,-0.906754,-0.37627,-0.32840,-0.86765,-0.825554,-0.94370,-0.26443,-0.31511,-0.80275,-0.89188,-1.00067,-0.18447,-0.32266,-0.74658,-0.76385,-0.790394,-0.44128,-0.09273,-0.66864,-0.67805,-0.66688,-0.632410,-0.46476,-0.72241,-0.70114,-0.69990,-0.614551,-0.70583,-0.78319,-0.81516,-0.66161,-0.61003,-0.54520,-0.76262,-0.76332,-0.73906,-0.91023,-0.65073,-0.75918,-0.745980,-0.83794,-0.97689,-1.15663,-0.74943,-0.765008,-0.75539,-0.69971,-0.73479,4,0.002207,0.953449


In [ ]:
def mean_pressure_coef(act_df, impt_df):
  mean_act = list()
  mean_imp = list()
  for i in range(n):
      row_act = act_df.iloc[i,:]
      row_act_mean = row_act.values.mean()
      row_imp = impt_df.iloc[i,:]
      row_imp_mean = row_imp.values.mean()
      mean_act.append(row_act_mean)
      mean_imp.append(row_imp_mean)
  mean_act_ser = pd.Series(mean_act, name='actual-mean')
  mean_imp_ser = pd.Series(mean_imp, name='imputed-mean')
  mse_df = pd.concat([pd.Series(range(n),name='time'),mean_act_ser, mean_imp_ser],axis=1)
  mse_df.to_csv('knn3_Side_mean_Cp.csv', index=False)

In [ ]:
mean_pressure_coef(data.copy(), imp_df.copy() )

Saving the predicted output into a xlsx and Mean Squared error

In [ ]:
import time

ts = time.gmtime()

cur_datetime = time.strftime("%m-%d-%H:%M:%S", ts)

In [ ]:

pd_save.to_excel('knn_'+str(k)+'front_result.xlsx',index=False,sheet_name='k_value_'+str(k)) # results from KNN identtiying 

In [28]:
!pwd

/content/drive/My Drive/piv-project/knn_front
